## PHYS64 Spring 2024
### Lev Gruber

## Ammonia Local Computing Notebook

The goal of this file is to use a VQE to simulate the Symmetric Double Minima Potential (SDMP) of an Ammonia molecule.
We will primarily follow the literature presented in https://arxiv.org/pdf/2312.04230.pdf to do so.
Code was built from scratch using Qiskit documentation while working off ideas in this paper.

Steps:
1. Use the STO-6G basis and PySCF to run a restricted HF calculation of the Hamiltonian as described in (1) above.
2. Define a complete active space CAS (2e, 2o) and map the four-spin-orbital reduced Hamiltonian onto qubits using the Jordan-Wigner mapping.
3. Compute accuracy of the VQE by comparing to CASCI classical method of computation (or more simply the paper result).

Notes from paper:
- VQE simulation was using the state vector simulator, I'd like to see if I can use IBM experience to do this physically.
- They do use and get working the UCCSD ansatz which is easier for me to implement, but I would like to use their Givens circuit as well.


In [ ]:
# Import all necessary packages

# General packages / building molecule
import qiskit
import numpy as np
import matplotlib.pyplot as plt
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
import csv

# CAS packages
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.mappers import JordanWignerMapper

# Local simulation packages
from qiskit_aer import Aer #backend
from qiskit.primitives import Estimator

# VQE usage packages
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import L_BFGS_B #optimizer
from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

In [ ]:
'''

This block is the non-function version of what occurs with find_energy and nh3_sweep below, built before that as a test of concept.

'''


# Use PySCF to compute 'one-body and two-body integrals in electronic orbital basis'.. or more simply build the particle
driver = PySCFDriver(
    atom = 'N .0 .0 -0.4; H .0 .66 .0; H .57 -.33 .0; H -.57 -.33 .0',     # this is just for an arbitrary equilateral triangle tho so we may need to update
    unit=DistanceUnit.ANGSTROM,         # define the above distances to be in terms of Angstroms (meaning for ex. .5 on N is 5*10^-11 m)
    basis='sto6g' #figure out what this is
    
)

problem = driver.run()

# define 2e 2o transformer and use it to redefine problem w/ CAS(2o, 2e)
transformer = ActiveSpaceTransformer(2, 2)
as_problem = transformer.transform(problem)

# set up mapper and VQE
mapper = JordanWignerMapper() 
optimizer = L_BFGS_B() #Limited-memory BFGS Bound optimizer, goal to minimize value of differentiable scalar f

# setup the estimator primitive for the VQE
backend = Aer.get_backend('statevector_simulator')
estimator = Estimator()

# setup VQE using Hartree-Fock as ansatz
ansatz = UCCSD(
    as_problem.num_spatial_orbitals,
    as_problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        as_problem.num_spatial_orbitals,
        as_problem.num_particles,
        mapper
    )
)

vqe = VQE(estimator, ansatz, optimizer)


# ensure that the optimizer starts in the all-zero state which corresponds to the Hartree-Fock starting point
vqe.initial_point = [0] * ansatz.num_parameters

# Solve VQE 
vqe_solver = GroundStateEigensolver(mapper, vqe)
vqe_result = vqe_solver.solve(as_problem)
vqe.formatting_precision = 6

# Solve exact result
numpy_solver = NumPyMinimumEigensolver()
exact_calc = GroundStateEigensolver(mapper, numpy_solver)
exact_result = exact_calc.solve(as_problem)

print(vqe_result)
print(exact_result)

In [ ]:
# This block combines the above code into a loop system that returns a graph as the Nitrogen atom moves in the z-plane
temp_backend = Aer.get_backend('statevector_simulator')
def find_energy(problem, backend, mapper = JordanWignerMapper(), optimizer = L_BFGS_B()):
    '''
    Uses supplied parameters to run VQE on chosen backend and solve exact solution w/NumPyMinimumEigensolver
    
    Parameters:
    problem (PySCF Molecule): PySCF created particle
    rest: optional, see code below for uses
    
    Returns: 
    vqe energy (float64), exact energy (float64)
    '''
    # define 2e 2o transformer and use it to redefine problem w/ CAS(2o, 2e)
    transformer = ActiveSpaceTransformer(2, 2)
    as_problem = transformer.transform(problem)
    
    # create ansatz
    ansatz = UCCSD(
    as_problem.num_spatial_orbitals,
    as_problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        as_problem.num_spatial_orbitals,
        as_problem.num_particles,
        mapper
        )
    )
    
    # set up VQE with chosen backend an ensure it starts in an all-zero state corresponding to HF starting point
    estimator = Estimator()
    vqe = VQE(estimator, ansatz, optimizer)
    vqe.initial_point = [0] * ansatz.num_parameters

    # solve VQE 
    solver = GroundStateEigensolver(mapper, vqe)
    vqe_result = solver.solve(as_problem)
    vqe.formatting_precision = 6
    
    
    # solve exact result for comparison
    numpy_solver = NumPyMinimumEigensolver()
    exact_calc = GroundStateEigensolver(mapper, numpy_solver)
    exact_result = exact_calc.solve(problem)    
    
    return vqe_result.total_energies[0], exact_result.total_energies[0]



def nh3_sweep(distances, mapper = JordanWignerMapper(), optimizer = L_BFGS_B()):
    '''
    Uses find_energy at various defined nitrogen locations of NH3
    
    Parameters:
    distances (array): location of N in NH3 above or below x-y plane
    rest, optional and see VQE documentation for use
    
    Returns:
    vqe energies at each distance (array), exact energy at each distance (array)
    '''
    backend = Aer.get_backend('statevector_simulator')
    
    # create arrays to return
    vqe_energies = []
    exact_energies = []
    
    # define molecule -- nh3 as equilateral planar H3 and N above.
    molecule = 'N 0.0 0.0 {0}; H -0.8121 -0.4689 0.0; H 0.8121 -0.4689 0.0; H 0.0 0.9377 0.0' 

    
    # find the vqe and exact energy at each distance provided
    for i, d in enumerate(distances):
        atom = molecule.format(d)
        driver = PySCFDriver(
        atom,     # this is just for an arbitrary equilateral triangle tho so we may need to update
        unit=DistanceUnit.ANGSTROM,         # define the above distances to be in terms of Angstroms (meaning for ex. .5 on N is 5*10^-11 m)
        basis='sto6g' 
        )
        problem = driver.run()
        
        vqe_result, exact_result = find_energy(problem, backend)
        vqe_energies.append(vqe_result)
        exact_energies.append(exact_result)
        
        print('vqe energy is', vqe_result, 'at d=', d)
        
        
    return vqe_energies, exact_energies

In [ ]:
# This block uses the above functions to develop a plot of nitrogen atom distance to energy

# Define the range of distances for nitrogen above, below, and in the middle of hydrogens
distances = [-0.8, -0.7, -0.6, -0.55, -0.5, -0.45, -0.4, -0.35, -0.25, -0.1, 0.0, 0.1, 0.25, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.7, 0.8] 
vqe_energies, exact_energies = nh3_sweep(distances)

# Save data in a csv
# Transpose lists to match csv format
rows = zip(distances, vqe_energies, exact_energies)
with open('local_nh3_4272024.csv', 'w') as f: #INSERTBACKEND_INSERTDISTANCES_INSERTDATE
    writer = csv.writer(f)
    # Write the headers
    writer.writerow(['Distances', 'VQE Energies', 'Exact Energies'])
    # Write the data
    for row in rows:
        writer.writerow(row)
        
plt.plot(distances, exact_energies, label = 'Exact Solution')
plt.plot(distances, vqe_energies, 'x', label = 'VQE Solution')

plt.xlabel(r' Nitrogen Distance ($A^\circ$)')
plt.ylabel('Energy (Ha)')
plt.title('Ammonia SDMP')
plt.legend(loc = 'upper right')
